In [1]:
import tensorflow as tf
import numpy as np
import sklearn as sk
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.models import load_model

# Load Data

In [2]:
df = pd.read_csv('competition_data/Luke_hair_loss_documentation.csv')
df

,date,hair_loss,stay_up_late,pressure_level,coffee_consumed,brain_working_duration,school_assesssment,stress_level,shampoo_brand,swimming,hair_washing,hair_grease,dandruff,libido
0,1/01/2021,Few,2,Low,0,1,None,Low,Pantene,No,Y,3.0,None,1
1,2/01/2021,Few,0,Low,0,3,None,Low,Pantene,No,N,1.0,None,1
2,3/01/2021,Medium,3,Low,1,0,None,Low,Pantene,Yes,Y,2.0,None,2
3,4/01/2021,Few,2,Low,0,1,None,Low,Pantene,No,N,3.0,None,3
4,5/01/2021,Few,2,Low,0,1,None,Low,Pantene,No,Y,1.0,None,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,31/01/2022,Medium,1,Low,1,2,None,Low,Hair & Shoulder,No,N,1.0,None,5
396,1/02/2022,Few,1,Low,0,3,None,Low,Hair & Shoulder,Yes,Y,2.0,None,1
397,2/02/2022,Medium,1,Low,1,1,None,Low,Hair & Shoulder,No,N,2.0,None,5
398,3/02/2022,Medium,0,Low,1,1,None,Low,Hair & Shoulder,No,N,2.0,None,5


# Proccessing Data

In [3]:
df['dandruff'].replace('None',0,inplace=True)
df['dandruff'].replace('Few',1,inplace=True)
df['dandruff'].replace('Many',2, inplace=True)

df['hair_loss'].replace("Few", 0, inplace=True)
df['hair_loss'].replace("Medium", 0, inplace=True)
df['hair_loss'].replace("Many", 1, inplace=True)
df['hair_loss'].replace("A lot", 1, inplace=True)

df['swimming'].replace('Yes', 1, inplace=True)
df['swimming'].replace('No', 0, inplace=True)

df.replace('Low', 0, inplace=True)
df.replace('Medium', 1, inplace=True)
df.replace('High', 2, inplace=True)
df.replace('Very High', 3, inplace=True)

df['hair_washing'].replace('Y',1,inplace=True)
df['hair_washing'].replace('N',0,inplace=True)


df.head()


,date,hair_loss,stay_up_late,pressure_level,coffee_consumed,brain_working_duration,school_assesssment,stress_level,shampoo_brand,swimming,hair_washing,hair_grease,dandruff,libido
0,1/01/2021,0,2,0,0,1,None,0,Pantene,0,1,3.0,0,1
1,2/01/2021,0,0,0,0,3,None,0,Pantene,0,0,1.0,0,1
2,3/01/2021,0,3,0,1,0,None,0,Pantene,1,1,2.0,0,2
3,4/01/2021,0,2,0,0,1,None,0,Pantene,0,0,3.0,0,3
4,5/01/2021,0,2,0,0,1,None,0,Pantene,0,1,1.0,0,2


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date                    400 non-null    object 
 1   hair_loss               400 non-null    int64  
 2   stay_up_late            400 non-null    int64  
 3   pressure_level          400 non-null    int64  
 4   coffee_consumed         400 non-null    int64  
 5   brain_working_duration  400 non-null    int64  
 6   school_assesssment      400 non-null    object 
 7   stress_level            400 non-null    int64  
 8   shampoo_brand           400 non-null    object 
 9   swimming                400 non-null    int64  
 10  hair_washing            400 non-null    int64  
 11  hair_grease             396 non-null    float64
 12  dandruff                400 non-null    int64  
 13  libido                  400 non-null    int64  
dtypes: float64(1), int64(10), object(3)
memory

In [5]:
plot_cols = df.corr().loc['stay_up_late':, 'hair_loss'].abs().sort_values(ascending=False)
plot_cols

dandruff                  0.808751
stress_level              0.784986
pressure_level            0.779733
hair_grease               0.682490
coffee_consumed           0.671447
brain_working_duration    0.591093
stay_up_late              0.350551
swimming                  0.240211
libido                    0.140884
hair_washing              0.033312
Name: hair_loss, dtype: float64

In [6]:
X_df= df[['stress_level', 'pressure_level', 'dandruff' ,'coffee_consumed','brain_working_duration', 'stay_up_late', 'hair_washing']]

In [7]:
y_data = df['hair_loss'].to_numpy().astype(np.float32)
print(y_data.shape)

df.drop('hair_loss', axis=1, inplace=True)

X_data = X_df.to_numpy().astype(np.float32)

print(X_data.shape)

(400,)
(400, 7)


# Train Test Split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.1, random_state=42)
print(X_train.shape)
print(X_test.shape)
print('_____________')
print(y_train.shape)
print(y_test.shape)


(360, 7)
(40, 7)
_____________
(360,)
(40,)


# Model Building

In [9]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Input(shape=(7)))

model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['acc'], optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                512       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 3,137
Trainable params: 3,137
Non-trainable params: 0
_________________________________________________________________


# Training

In [10]:
model.fit(X_train, y_train,epochs=50,batch_size=64,validation_split=0.10, verbose=2)

Epoch 1/50


C:\Users\kakoo\anaconda3\envs\kev\lib\site-packages\keras\backend.py:5674: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits, "Sigmoid", "binary_crossentropy"


6/6 - 1s - loss: 0.6801 - acc: 0.8302 - val_loss: 0.6102 - val_acc: 0.8056 - 506ms/epoch - 84ms/step
Epoch 2/50
6/6 - 0s - loss: 0.5864 - acc: 0.8364 - val_loss: 0.5396 - val_acc: 0.8056 - 40ms/epoch - 7ms/step
Epoch 3/50
6/6 - 0s - loss: 0.5266 - acc: 0.8302 - val_loss: 0.4824 - val_acc: 0.8333 - 31ms/epoch - 5ms/step
Epoch 4/50
6/6 - 0s - loss: 0.4681 - acc: 0.8642 - val_loss: 0.4200 - val_acc: 0.8611 - 29ms/epoch - 5ms/step
Epoch 5/50
6/6 - 0s - loss: 0.4132 - acc: 0.8827 - val_loss: 0.3654 - val_acc: 0.8333 - 30ms/epoch - 5ms/step
Epoch 6/50
6/6 - 0s - loss: 0.3578 - acc: 0.8920 - val_loss: 0.3167 - val_acc: 0.8611 - 30ms/epoch - 5ms/step
Epoch 7/50
6/6 - 0s - loss: 0.3078 - acc: 0.8951 - val_loss: 0.2758 - val_acc: 0.8611 - 29ms/epoch - 5ms/step
Epoch 8/50
6/6 - 0s - loss: 0.2616 - acc: 0.9167 - val_loss: 0.2209 - val_acc: 0.9167 - 29ms/epoch - 5ms/step
Epoch 9/50
6/6 - 0s - loss: 0.2256 - acc: 0.9228 - val_loss: 0.1856 - val_acc: 0.9722 - 28ms/epoch - 5ms/step
Epoch 10/50
6/6 - 0

# Evaluating

In [11]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 1ms/step - loss: 0.0902 - acc: 0.9750


[0.09019243717193604, 0.9750000238418579]

# Result: He was hair loss!!!

In [12]:
model.predict(np.array([[1,1,3,1,2,1,1]]))



1/1 [==============================] - 0s 120ms/step


array([[0.95320255]], dtype=float32)

In [13]:
model.predict(np.array([[1,2,3,1,2,1,1]]))

1/1 [==============================] - 0s 35ms/step


array([[0.9784661]], dtype=float32)

In [14]:
model.predict(np.array([[3,2,3,1,2,1,1]]))

1/1 [==============================] - 0s 22ms/step


array([[0.99433655]], dtype=float32)

In [15]:
i=1

In [16]:
check_arr=[0,0,0,0,0,0,0,0,0,0]

In [17]:
model.predict(np.array([[3,2,3,1,2,1,i]]))

1/1 [==============================] - 0s 20ms/step


array([[0.99433655]], dtype=float32)

In [18]:
for i in range (5):
    if i == 0 :
         continue
    for j in range(5):
        if j==0 :
            continue
        for k in range(4):
            if k==0 :
                continue
            for m in range(4):
                if m==0:
                    continue
                for n in range(4):
                    if n ==0:
                        continue
                    for o in range(5):
                        if o ==0:
                            continue
                        for p in range(3):
                            if p==0:
                                continue
                            print(model.predict(np.array([[i,j,k,m,n,o,p]]))[0][0]*100)
                            if 0<=model.predict(np.array([[i,j,k,m,n,o,p]]))[0][0]*100<10:
                                check_arr[0]+=1
                            elif 10<=model.predict(np.array([[i,j,k,m,n,o,p]]))[0][0]*100<20:
                                check_arr[1]+=1
                            elif 20<=model.predict(np.array([[i,j,k,m,n,o,p]]))[0][0]*100<30:
                                check_arr[2]+=1
                            elif 30<=model.predict(np.array([[i,j,k,m,n,o,p]]))[0][0]*100<40:
                                check_arr[3]+=1
                            elif 40<=model.predict(np.array([[i,j,k,m,n,o,p]]))[0][0]*100<50:
                                check_arr[4]+=1
                            elif 50<=model.predict(np.array([[i,j,k,m,n,o,p]]))[0][0]*100<60:
                                check_arr[5]+=1
                            elif 60<=model.predict(np.array([[i,j,k,m,n,o,p]]))[0][0]*100<70:
                                check_arr[6]+=1
                            elif 70<=model.predict(np.array([[i,j,k,m,n,o,p]]))[0][0]*100<80:
                                check_arr[7]+=1
                            elif 80<=model.predict(np.array([[i,j,k,m,n,o,p]]))[0][0]*100<90:
                                check_arr[8]+=1
                            elif 90<=model.predict(np.array([[i,j,k,m,n,o,p]]))[0][0]*100<100:
                                check_arr[9]+=1

1/1 [==============================] - 0s 19ms/step
55.19304275512695
1/1 [==============================] - 0s 18ms/step
12.168747186660767
1/1 [==============================] - 0s 30ms/step
15.202203392982483
1/1 [==============================] - 0s 23ms/step
6.682341545820236
1/1 [==============================] - 0s 22ms/step
1.6644515097141266
1/1 [==============================] - 0s 22ms/step
0.7928892970085144
1/1 [==============================] - 0s 20ms/step
0.21300287917256355
1/1 [==============================] - 0s 31ms/step
0.10063712252303958
1/1 [==============================] - 0s 20ms/step
57.5969398021698
1/1 [==============================] - 0s 22ms/step
7.439236342906952
1/1 [==============================] - 0s 19ms/step
30.641737580299377
1/1 [==============================] - 0s 18ms/step
6.192922219634056
1/1 [==============================] - 0s 24ms/step
6.357133388519287
1/1 [==============================] - 0s 31ms/step
1.73724964261055
1/1 [========

In [19]:
print(check_arr)

[383, 170, 161, 155, 181, 157, 221, 246, 391, 1391]
